# Hit API

In [1]:
from caltrain import RwcSfTrains, convert_time_str_to_local_tz_timestamp, RWC_CALTRAIN_STOP_ID, format_for_display

In [3]:
{os.environ.pop(k) for k in os.environ.keys() if 'PROXY' in k}

{'http://127.0.0.1:8888',
 'https://goproxy.dev.bloomberg.com,direct',
 'localhost,.dev.bloomberg.com,query.di.secondmeasure.com'}

In [4]:
self = RwcSfTrains('north')
self.fetch_data()

redwood_city
san_francisco


In [5]:
self.next_train_options()

redwood_city
san_francisco


,departure stop,arrival stop,scheduled departure,late departing,scheduled arrival,late arriving,travel time
0,Redwood City Caltrain Station,4th & King,10:18,0 minutes,10:51,0 minutes,33 minutes


In [13]:
self.get_next_sf_trips_from_rwc()

Index(['stop_name', 'scheduled_departure', 'expected_departure', 'vehicle_id'], dtype='object')
Index(['stop_name', 'scheduled_departure', 'expected_departure', 'vehicle_id'], dtype='object')
Index(['stop_name', 'scheduled_departure', 'expected_departure', 'vehicle_id'], dtype='object')


AttributeError: 'DataFrame' object has no attribute 'expected_arrival'

In [15]:
(self.trains_with_departure_stop.merge(self.get_vehicle_onward_stops(), on="vehicle_id", how="left")
    .drop(columns=["vehicle_id"]))

Index(['stop_name', 'scheduled_departure', 'expected_departure', 'vehicle_id'], dtype='object')


,stop_name_x,scheduled_departure_x,expected_departure_x,time_late,stop_name_y,scheduled_departure_y,expected_departure_y
0,Redwood City Caltrain Station,2023-06-09 10:18:00-07:00,2023-06-09 10:18:00-07:00,0 days 00:00:00,Sunnyvale Caltrain Station,2023-06-09 09:56:00-07:00,2023-06-09 09:56:15-07:00
1,Redwood City Caltrain Station,2023-06-09 10:18:00-07:00,2023-06-09 10:18:00-07:00,0 days 00:00:00,Mountain View Caltrain Station,2023-06-09 10:01:00-07:00,2023-06-09 10:01:00-07:00
2,Redwood City Caltrain Station,2023-06-09 10:18:00-07:00,2023-06-09 10:18:00-07:00,0 days 00:00:00,Palo Alto Caltrain Station,2023-06-09 10:09:00-07:00,2023-06-09 10:09:00-07:00
3,Redwood City Caltrain Station,2023-06-09 10:18:00-07:00,2023-06-09 10:18:00-07:00,0 days 00:00:00,Menlo Park Caltrain Station,2023-06-09 10:12:00-07:00,2023-06-09 10:12:30-07:00
4,Redwood City Caltrain Station,2023-06-09 10:18:00-07:00,2023-06-09 10:18:00-07:00,0 days 00:00:00,Redwood City Caltrain Station,2023-06-09 10:18:00-07:00,2023-06-09 10:18:00-07:00
5,Redwood City Caltrain Station,2023-06-09 10:18:00-07:00,2023-06-09 10:18:00-07:00,0 days 00:00:00,Hillsdale Caltrain Station,2023-06-09 10:25:00-07:00,2023-06-09 10:25:45-07:00
6,Redwood City Caltrain Station,2023-06-09 10:18:00-07:00,2023-06-09 10:18:00-07:00,0 days 00:00:00,San Mateo Caltrain Station,2023-06-09 10:31:00-07:00,2023-06-09 10:31:00-07:00
7,Redwood City Caltrain Station,2023-06-09 10:18:00-07:00,2023-06-09 10:18:00-07:00,0 days 00:00:00,Millbrae Caltrain Station,2023-06-09 10:38:00-07:00,2023-06-09 10:38:15-07:00
8,Redwood City Caltrain Station,2023-06-09 10:18:00-07:00,2023-06-09 10:18:00-07:00,0 days 00:00:00,22nd Street Caltrain Station,2023-06-09 10:51:00-07:00,2023-06-09 10:51:15-07:00
9,Redwood City Caltrain Station,2023-06-09 10:18:00-07:00,2023-06-09 10:18:00-07:00,0 days 00:00:00,4th & King,0 days 00:06:00.000000503,0 days 00:06:00.000000503


# Misc info gathering

In [13]:
CALTRAIN_OPERATOR_ID = "CT"
# This gives caltrain operatorID: CT
operators = f"http://api.511.org/transit/operators?api_key={os.environ['CALTRAIN_API_KEY']}"
# this gives the name of rwc caltrain stop
stops = f"http://api.511.org/transit/stops?api_key={os.environ['CALTRAIN_API_KEY']}&operator_id={CALTRAIN_OPERATOR_ID}&format=json"

In [17]:
response_stops = RwcSfTrains.request_to_dict(stops)
# [x for x in response_stops.json()['Contents']['dataObjects']['ScheduledStopPoint'] if 'Redwood'.lower() in x['Name'].lower()]

In [24]:
self.departures_response.keys()#['redwood_city']

dict_keys(['redwood_city', 'san_francisco', '22nd_street'])

In [25]:
self.departures_response['san_francisco']

{'ServiceDelivery': {'ResponseTimestamp': '2023-06-08T15:54:04Z',
  'ProducerRef': 'CT',
  'Status': True,
  'StopMonitoringDelivery': {'version': '1.4',
   'ResponseTimestamp': '2023-06-08T15:54:04Z',
   'Status': True,
   'MonitoredStopVisit': [{'RecordedAtTime': '2023-06-08T15:53:52Z',
     'MonitoringRef': '70142',
     'MonitoredVehicleJourney': {'LineRef': 'L4',
      'DirectionRef': 'S',
      'FramedVehicleJourneyRef': {'DataFrameRef': '2023-06-08',
       'DatedVehicleJourneyRef': '406'},
      'PublishedLineName': 'LTD 4',
      'OperatorRef': 'CT',
      'OriginRef': '70012',
      'OriginName': 'San Francisco Caltrain Station',
      'DestinationRef': '70262',
      'DestinationName': 'San Jose Diridon',
      'Monitored': True,
      'InCongestion': None,
      'VehicleLocation': {'Longitude': '-122.234032', 'Latitude': '37.487381'},
      'Bearing': None,
      'Occupancy': None,
      'VehicleRef': '406',
      'MonitoredCall': {'StopPointRef': '70142',
       'StopPoint

In [25]:
response_stops['Contents']['dataObjects']['ScheduledStopPoint'][2]

{'id': '70021',
 'Extensions': {'LocationType': '0',
  'PlatformCode': None,
  'ParentStation': '22nd_street'},
 'Name': '22nd Street Caltrain Station',
 'Location': {'Longitude': '-122.39188', 'Latitude': '37.757599'},
 'Url': None,
 'StopType': 'onstreetBus'}

In [19]:
[x for x in response_stops['Contents']['dataObjects']['ScheduledStopPoint'] if 'San' in x['Name']]

NameError: name 'response_stops' is not defined

In [51]:
self.departures_response["ServiceDelivery"]["StopMonitoringDelivery"]["MonitoredStopVisit"]

[{'RecordedAtTime': '1970-01-01T00:00:00Z',
  'MonitoringRef': '70012',
  'MonitoredVehicleJourney': {'LineRef': 'L3',
   'DirectionRef': 'S',
   'FramedVehicleJourneyRef': {'DataFrameRef': '2023-06-08',
    'DatedVehicleJourneyRef': '302'},
   'PublishedLineName': 'LTD 3',
   'OperatorRef': 'CT',
   'OriginRef': '70012',
   'OriginName': 'San Francisco Caltrain Station',
   'DestinationRef': '70262',
   'DestinationName': 'San Jose Diridon',
   'Monitored': True,
   'InCongestion': None,
   'VehicleLocation': {'Longitude': '', 'Latitude': ''},
   'Bearing': None,
   'Occupancy': None,
   'VehicleRef': None,
   'MonitoredCall': {'StopPointRef': '70012',
    'StopPointName': 'San Francisco Caltrain Station',
    'VehicleLocationAtStop': '',
    'VehicleAtStop': '',
    'DestinationDisplay': 'San Jose Diridon',
    'AimedArrivalTime': '2023-06-08T13:27:00Z',
    'ExpectedArrivalTime': None,
    'AimedDepartureTime': '2023-06-08T13:27:00Z',
    'ExpectedDepartureTime': '2023-06-08T13:27:0

In [47]:
[x for x in response_stops['Contents']['dataObjects']['ScheduledStopPoint'] if 'Redwood' in x['Name']]

[{'id': 'redwood_city',
  'Extensions': {'LocationType': '1',
   'PlatformCode': None,
   'ParentStation': None},
  'Name': 'Redwood City',
  'Location': {'Longitude': '-122.2315', 'Latitude': '37.485865'},
  'Url': None,
  'StopType': 'onstreetBus'},
 {'id': '70141',
  'Extensions': {'LocationType': '0',
   'PlatformCode': None,
   'ParentStation': 'redwood_city'},
  'Name': 'Redwood City Caltrain Station',
  'Location': {'Longitude': '-122.231936', 'Latitude': '37.486159'},
  'Url': None,
  'StopType': 'onstreetBus'},
 {'id': '70142',
  'Extensions': {'LocationType': '0',
   'PlatformCode': None,
   'ParentStation': 'redwood_city'},
  'Name': 'Redwood City Caltrain Station',
  'Location': {'Longitude': '-122.232', 'Latitude': '37.486101'},
  'Url': None,
  'StopType': 'onstreetBus'},
 {'id': 'RC',
  'Extensions': {'LocationType': '0',
   'PlatformCode': None,
   'ParentStation': None},
  'Name': 'Temporary Stop - Redwood City',
  'Location': {'Longitude': '-122.2320244337', 'Latitude

In [50]:
[x for x in response_stops['Contents']['dataObjects']['ScheduledStopPoint'] if 'San Fr' in x['Name']]

[{'id': 'san_francisco',
  'Extensions': {'LocationType': '1',
   'PlatformCode': None,
   'ParentStation': None},
  'Name': 'San Francisco Caltrain Station',
  'Location': {'Longitude': '-122.394911', 'Latitude': '37.776404'},
  'Url': None,
  'StopType': 'onstreetBus'},
 {'id': '70011',
  'Extensions': {'LocationType': '0',
   'PlatformCode': None,
   'ParentStation': 'san_francisco'},
  'Name': 'San Francisco Caltrain Station',
  'Location': {'Longitude': '-122.394992', 'Latitude': '37.77639'},
  'Url': None,
  'StopType': 'onstreetBus'},
 {'id': '70012',
  'Extensions': {'LocationType': '0',
   'PlatformCode': None,
   'ParentStation': 'san_francisco'},
  'Name': 'San Francisco Caltrain Station',
  'Location': {'Longitude': '-122.394935', 'Latitude': '37.776348'},
  'Url': None,
  'StopType': 'onstreetBus'},
 {'id': 'south_sf',
  'Extensions': {'LocationType': '1',
   'PlatformCode': None,
   'ParentStation': None},
  'Name': 'South San Francisco Caltrain Station',
  'Location': {'